### 기본 세팅
작동 시간이 너무 빠르면 자동툴로 인식하는 경우가 있어 계속 sleep을 걸음

In [1]:
import os
import time
import pandas as pd

from selenium import webdriver
import pyperclip
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup 

In [2]:
# 셀레니움 드라이버로 웹 실행
driver = webdriver.Chrome('chromedriver.exe')
driver.implicitly_wait(3)

In [3]:
# 클립보드에 input을 복사하고, ActionChain을 이용해 복사 내용을 로그인 폼에 붙여넣기
def copy_input(xpath, input):
    pyperclip.copy(input)
    driver.find_element_by_xpath(xpath).click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)

In [4]:
# 네이버 id & pw 입력
naver_id = ''
naver_pw = ''

In [5]:
# 네이버 로그인 
driver.get('https://nid.naver.com/nidlogin.login')             
time.sleep(1)

copy_input('//*[@id="id"]', naver_id)
time.sleep(1)
copy_input('//*[@id="pw"]', naver_pw)
time.sleep(1)

driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()

### 주소 및 닉네임 크롤링 테스트
INFP & ENFP 게시판: menuid=18

In [6]:
page = 0
address = []
nickname = []

while page <= 1:
    try:
        # 페이지 및 iframe 전환
        page += 1
        driver.get(f"https://cafe.naver.com/ArticleList.nhn?search.clubid=11856775&search.menuid=18&search.boardtype=L&search.page={page}")
        driver.switch_to.frame("cafe_main")
        
        # 더이상 게시글이 없을 경우
        try:
            if driver.find_element_by_class_name("nodata").text == '등록된 게시글이 없습니다.':
                break
        except:
            pass
        
        # 페이지당 주소 및 닉네임 20개 크롤링
        for i in range(1, 31, 1):
            address.append(driver.find_element_by_xpath(f"/html/body/div[1]/div/div[4]/table/tbody/tr[{i}]/td[1]/div[2]/div/a[1]").get_attribute("href"))
            nickname.append(driver.find_element_by_xpath(f"/html/body/div[1]/div/div[4]/table/tbody/tr[{i}]/td[2]/div/table/tbody/tr/td/a").get_attribute("onclick"))
                            
        # 기존 웹페이지로 복귀
        driver.switch_to.default_content()   
        
    except:
        pass

In [7]:
# 크롤링한 게시글 주소
address

['https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504815&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504799&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504783&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504778&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504759&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504751&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504738&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype

In [8]:
len(address)

30

In [9]:
# 크롤링한 닉네임이 포함되어 있는 onclick 타입
nickname

["ui(event, 'gina4148',3,'누가봐도Entp','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'gggg122',3,'Infp아양이','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'dlforud1523',3,'포에트리 ESFP','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'llrhtnll',3,'주시쿨좋아하는사람','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'koe1012',3,'INFP흰여우','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'y2032ryu',3,'한명의인프피','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'djwls222',3,'오징징어 enfp','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'k0555',3,'찐찐infp','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'heesong04',3,'국화빵ENFP','11856775','me', 'false', 'true', 'mbticafe', 'false', '1

In [10]:
len(nickname)

30

In [11]:
nickname[0]

"ui(event, 'gina4148',3,'누가봐도Entp','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;"

In [12]:
type(nickname[0])

str

### onclick 타입 nickname에서 mbti 추출

In [13]:
mbti_group = ['intj', 'intp', 'entj', 'entp',
              'infj', 'infp', 'enfj', 'enfp',
              'istj', 'isfj', 'estj', 'esfj',
              'istp', 'isfp', 'estp', 'esfp']
len(mbti_group)

16

In [14]:
# onclick 타입 nickname 소문자화
for i in range(len(nickname)):
    nickname[i] = nickname[i].lower()

In [15]:
drop_idx = []   

for i in range(len(nickname)):
    check_cnt = 0
    
    for mbti in mbti_group:        
        # 닉네임에 mbti가 포함된 경우 닉네임을 mbti화
        if mbti in nickname[i]:
            nickname[i] = mbti
        
        # 닉네임에 mbti가 포함되지 않은 경우
        if mbti not in nickname[i]:
            check_cnt += 1
        
    # 닉네임에 mbti가 아예 없을 경우 해당 인덱스 저장
    if check_cnt == 16:
        drop_idx.append(i)

In [16]:
nickname

['entp',
 'infp',
 'esfp',
 "ui(event, 'llrhtnll',3,'주시쿨좋아하는사람','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'infp',
 "ui(event, 'y2032ryu',3,'한명의인프피','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'enfp',
 'infp',
 'enfp',
 'enfp',
 'entp',
 'enfp',
 "ui(event, 'kmj90233',3,'엔프피인간','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'dkfhddl726',3,'치킨나라떡볶이공주','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'isfj',
 "ui(event, 'daldaldaily',3,'잇팁인데여','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'enfj',
 "ui(event, 'astros29',3,'뽀로','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'astros29',3,'뽀로','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'eos_t_ld',3,'익명9','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;"

In [17]:
drop_idx   # 닉네임에 mbti가 포함되지 않아 drop할 인덱스

[3, 5, 12, 13, 15, 17, 18, 19, 21, 22, 25]

In [18]:
len(drop_idx)

11

### 기존 address, nickname에서 drop_idx에 해당되면 드랍

In [19]:
print(' drop전 address 개수: {0}'.format( len(address) ))
print(' drop전 nickname 개수: {0}'.format( len(nickname) ))

 drop전 address 개수: 30
 drop전 nickname 개수: 30


In [20]:
for i in range(len(address)):
    
    while i in drop_idx:
        address.pop(i)
        nickname.pop(i)
        drop_idx.remove(i)
        
        for idx in range(len(drop_idx)):
            drop_idx[idx] -= 1
    
print(' drop후 address 개수: {0}'.format( len(address) ))
print(' drop후 nickname 개수: {0}'.format( len(nickname) ))

 drop후 address 개수: 19
 drop후 nickname 개수: 19


In [21]:
nickname

['entp',
 'infp',
 'esfp',
 'infp',
 'enfp',
 'infp',
 'enfp',
 'enfp',
 'entp',
 'enfp',
 'isfj',
 'enfj',
 'enfp',
 'infp',
 'enfj',
 'enfp',
 'enfp',
 'isfj',
 'isfj']

### address, nickname으로 게시글 제목과 본문 추출

In [34]:
title = []
content = []

for url in address:
    # 페이지 및 iframe 전환
    driver.get(url)
    driver.switch_to.frame("cafe_main")
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 제목과 본문 크롤링
    cur_title = driver.find_element_by_css_selector("h3.title_text").text
    cur_contents = soup.find("body").find_all("p")
    #cur_contents = driver.find_element_by_xpath(f"/html/body/div/div/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[1]/div/div/div/p").text
    #cur_contents = soup.find("p").find_all("span")

    title.append(cur_title)
    content.append(cur_contents)

In [ ]:
/html/body/div/div/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[1]/div/div/div/p[2]/span
/html/body/div/div/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[1]/div/div/div/p[3]/span

/html/body/div/div/div/div[2]/div[2]/div[1]/div/div/p/text()[1]
/html/body/div/div/div/div[2]/div[2]/div[1]/div/div/p/text()[2]

/html/body/div/div/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[1]/div/div/div/p/span

In [35]:
# 크롤링한 게시글 제목
title

['엔프피 질문드립니다',
 '썸기간 연락문제ㅠㅠㅜㅠ',
 '엔프피 여자 질문..',
 '내 키보드 고장났댜아아아~입미댜아아...',
 '인프피가 장난 많이쳐요?ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 'infp 분들~~',
 'ENFP 막내동생 싸우고 난 뒤 쪽지를 줬어요',
 '역시 엔프피는 직진이었나봐요',
 '인프피분들 애니어그램 7번이 나오시나요??',
 '풀리지 않는 질문 여자와 남자는 친구가 될수있는가',
 '인프피 남자 고민 (도와주세요..)',
 '잉뿌피 친구가 한 말✨',
 '요즘 유행하는 ai 필터 써봤는데용!!',
 '뒤숭숭하고 잠도 안와서 INFP 질문이나 받아엽^^',
 '여러분 여러분 야라분 여러버ㅜㄴ 인프피 분글 제발요 제발요 제발요',
 '응팔 mbti 추측',
 'enfp/infp 여러분들 재밌게 본 드라마,영화 추천 좀 해주세요',
 'ENFP분들 적극적인? 여자 어떠시나요',
 'ENFP와 연락']

In [36]:
# 크롤링한 게시글 본문
content

[[],
 [<p class="text">
              이 게시글을 카페 캘린더에서 제외하시겠습니까?
          </p>,
  <p class="text">
              이 게시글을 카페 캘린더로 보내시겠습니까?
          </p>,
  <p class="se-text-paragraph se-text-paragraph-align-" id="SE-23fa68e3-7ae8-4731-a93b-63caff1c5ae9" style=""><span class="se-fs- se-ff-" id="SE-404f80de-99c4-40a2-9e25-293ea48f7437" style="">좀 긴데 제발 읽어주세여ㅜ</span></p>,
  <p class="se-text-paragraph se-text-paragraph-align-" id="SE-9e487733-00c2-4f91-9ed3-12b14da826a5" style=""><span class="se-fs- se-ff-" id="SE-cc51555b-0b1d-4c02-851c-8041d76ebc73" style="">​</span></p>,
  <p class="se-text-paragraph se-text-paragraph-align-" id="SE-8eae3828-17c5-454e-afa3-fa4d9813e8ab" style=""><span class="se-fs- se-ff-" id="SE-a954b132-628f-4bc6-8795-5a18cd7a419c" style="">​</span></p>,
  <p class="se-text-paragraph se-text-paragraph-align-" id="SE-9e601caf-74d0-497b-9760-90577ed009c1" style=""><span class="se-fs- se-ff-" id="SE-30ef6f89-8875-45e0-ae50-a6f26f1893d0" style="">저는 진짜 연락에 집착없는 편이라서ㅜㅠ</s

In [ ]:
#data_crawled_from_naver_cafe = pd.DataFrame({'mbti': nickname, 'title': title, 'content': content})